<a href="https://colab.research.google.com/github/cheatham1/BackUp/blob/main/simpleT5_trainPubMedAbstractsQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install evaluate
!pip install transformers
#!pip install transformers==4.16.2
#!pip install sentencepiece
#!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#!pip install python-Levenshtein

In [ ]:
#import evaluate
import transformers
#import torch
#from transformers import pipeline
from transformers import TrainingArguments, Trainer

In [ ]:
!pip install simplet5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_qa = pd.read_csv('https://raw.githubusercontent.com/cheatham1/EU-JAV/main/QuesionsAndAnswers.tsv',
                   sep='\t',
                   names=["index","question","answer","pmid"],
                   header=0,
                   index_col = [0]

)
df_qa.head()

,question,answer,pmid
index,,,
197,How much did overall positive sentiment increa...,+1.1 points,36064471
543,What scores did the transformer-based model in...,"0.816, 0.668, 0.528, and 0.743",36002778
530,What was the threshold for specificity?,0.9,36004803
276,What was the micro-AUROC for primary lung canc...,0.913 and 0.946,36050674
498,What was the range of the AUROC of the rule-ba...,0.92 to 0.96,36010232


In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/cheatham1/EU-JAV/main/100nlp_pubmed_articles.tsv',
                   sep='\t',
                   names=["index","pmid","Title","Abstract","Year","Journal"],
                   header=0,
                   index_col = [0]
)
data.head(2)

,pmid,Title,Abstract,Year,Journal
index,,,,,
0,36085896,Using Natural Language Processing of Clinical ...,Potential of natural language processing (NLP)...,2022,Annu Int Conf IEEE Eng Med Biol Soc
1,36085766,A streamable large-scale clinical EEG dataset ...,Deep Learning has revolutionized various field...,2022,Annu Int Conf IEEE Eng Med Biol Soc


In [ ]:
data_abstract = data[["pmid","Abstract"]]
data_abstract.head(2)

,pmid,Abstract
index,,
0,36085896,Potential of natural language processing (NLP)...
1,36085766,Deep Learning has revolutionized various field...


In [ ]:
df = pd.merge(df_qa, data_abstract)
df.head()

,question,answer,pmid,Abstract
0,How much did overall positive sentiment increa...,+1.1 points,36064471,Closer integration of health and social care i...
1,What was the AFINN score for reviews containin...,2.3 points,36064471,Closer integration of health and social care i...
2,What is a policy priority in many countries?,Closer integration of health and social care,36064471,Closer integration of health and social care i...
3,What was used to examine whether experiencing ...,Difference-in-difference analysis,36064471,Closer integration of health and social care i...
4,What did we combine with supervised machine le...,Natural Language Processing,36064471,Closer integration of health and social care i...


In [ ]:
df.shape

(582, 4)

In [ ]:
df.replace("\n", "", inplace = True)
df.replace(":", "", inplace = True)

In [ ]:
# T5 model expects source_text and target_text as column headings
df = df.rename(columns={"answer":"target_text"})

df["source_text"] = df['question'] + " : " + df["Abstract"]

df = df[['source_text', 'target_text']]

# T5 model expects a task related prefix: add prefix "summarize: "
#df['source_text'] = "summarize: " + df['source_text']


In [ ]:
df.iloc[0,0]

"How much did overall positive sentiment increase during the COVID-19 period? : Closer integration of health and social care is a policy priority in many countries. The COVID-19 pandemic has reinforced the necessity of joining up health and social care systems, especially in care home settings. However, the meaning and perceived importance of integration for residents' and carers' experience is unclear and we do not know whether it has changed during the pandemic. Using unique data from on-line care home service reviews, we combined multiple methods. We used Natural Language Processing with supervised machine learning to construct a measure of sentiment for care home residents' and their relatives' (measured by AFINN score). Difference-in-difference analysis was used to examine whether experiencing integrated care altered these sentiments by comparing changes in sentiment in reviews related to integration (containing specific terms) to those which were not. Finally, we used network ana

In [ ]:
from sklearn.model_selection import train_test_split

train_df, evaltest_df,  = train_test_split(df, train_size=0.6)
eval_df, test_df,  = train_test_split(evaltest_df, train_size=0.5)

In [ ]:
from simplet5 import SimpleT5
model = SimpleT5()

model.from_pretrained("t5","t5-base")

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

In [ ]:
max_input_length = 512
max_target_length = 150
batch_size = 8
epochs = 3

model.train(train_df=train_df,
            eval_df =eval_df,
            source_max_token_len= max_input_length,
            target_max_token_len= max_target_length,
            batch_size=batch_size,
            max_epochs=epochs,
            use_gpu=True,
            outputdir= "outputs",
            early_stopping_patience_epochs= 0)


INFO:pytorch_lightning.utilities.distributed:GPU available: True, used: True
INFO:pytorch_lightning.utilities.distributed:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.distributed:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.accelerators.gpu:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0         Non-trainable params
222 M     Total params
891.614   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
INFO:pytorch_lightning.utilities.seed:Global seed set to 42
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
!ls -lrt outputs

total 12
drwxr-xr-x 2 root root 4096 Sep 29 14:28 simplet5-epoch-0-train-loss-0.6735-val-loss-0.2103
drwxr-xr-x 2 root root 4096 Sep 29 14:29 simplet5-epoch-1-train-loss-0.1379-val-loss-0.2377
drwxr-xr-x 2 root root 4096 Sep 29 14:30 simplet5-epoch-2-train-loss-0.0893-val-loss-0.2384


In [ ]:
stop here

SyntaxError: ignored

In [ ]:
# load the trained model
model.load_model("outputs/simplet5-epoch-2-train-loss-0.0725-val-loss-0.2368", use_gpu=True)


In [ ]:
test_source_text = test_df[['source_text']].values.tolist()
test_target_text = test_df[['target_text']].values.tolist()

In [ ]:
text = test_df.iloc[0,0]
text

In [ ]:
model.predict(text)

In [ ]:
target_text = test_df.iloc[0,1]
target_text

In [ ]:
# I am sure there is a better way of doing this
answers=[]

for question in test_source_text:

  answer = model.predict(question[0])
  answers.append(answer)


In [ ]:
import sklearn.metrics
from sklearn.metrics import classification_report

In [ ]:
summary_df = test_df.copy()
summary_df['prediction'] = np.array(answers)
summary_df.head()

In [ ]:
#from Levenshtein import distance as lev

#summary_df['ldist'] = lev(str(summary_df['target_text']),str(summary_df['prediction']))

In [ ]:
#summary_df["lev_score"] = summary_df.apply(lambda x: lev(x.prediction,x.target_text), axis=1)
#summary_df['string_match'] = summary_df.apply(lambda x: x.prediction in x.target_text, axis=1)
#summary_df['string_match'] = summary_df.apply(lambda x: x.prediction in x.target_text or x.target_text in x.prediction, axis=1)

In [ ]:
#summary_df['success'] = summary_df['lev_score']<=15


In [ ]:
summary_df.head(50)

In [ ]:
#summary_df['success'].value_counts()

In [ ]:
#print(train_df.shape, eval_df.shape, test_df.shape)

In [ ]:
#summary_df['population size'] = df["target_text"].str.contains("population", case=False, na=False).astype(int)
#summary_df['population size'].value_counts()

In [ ]:
#summary_df['effect size'] = df["target_text"].str.contains("effect" or "result", case=False, na=False).astype(int)
#summary_df['effect size'].value_counts()

In [ ]:
#summary_df['data'] = df.apply(lambda row: row.astype(str).str.contains('dataset').any(), axis=1)
#summary_df['data'].value_counts()